# Popes
--------------
By Justin Tung

data from: 

https://en.wikipedia.org/wiki/List_of_popes

https://www.theguardian.com/news/datablog/2013/feb/13/popes-full-list

In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from matplotlib import rcParams

In [2]:
df = pd.read_excel('data/popes.xlsx')
df['years_of_reign']= (df['End_date'] - df['Start_date'])



In [3]:
pope_count = df.shape[0]
family_count = len(df['Family'].unique())
family_count

8

In [4]:
years_df = df[['English_name', 'Start_date', 'End_date', 'years_of_reign', 'Family', 'Order']]
years_df.insert(0, 'year_node', range(len(df)))
families = list(years_df['Family'].unique())
orders = list(years_df['Order'].unique())
for count, family in enumerate(families, start=pope_count):
    years_df.loc[years_df['Family'] == family, 'family_node'] = int(count)
for count, order in enumerate(orders, start=pope_count+family_count):
    years_df.loc[years_df['Order'] == order, 'order_node'] = int(count)

years_df['family_node'] = years_df['family_node'].fillna(pope_count)
years_df['order_node'] = years_df['order_node'].fillna(pope_count+family_count)
years_df['names_and_years'] = years_df.apply(lambda row: f"{row['English_name']} ({row['Start_date']} - {row['End_date']})", axis=1)

/var/folders/6_/k0_nmk_j3c766qsynb7pr0h40000gp/T/ipykernel_33019/3840281498.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  years_df.loc[years_df['Family'] == family, 'family_node'] = int(count)
/var/folders/6_/k0_nmk_j3c766qsynb7pr0h40000gp/T/ipykernel_33019/3840281498.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  years_df.loc[years_df['Order'] == order, 'order_node'] = int(count)
/var/folders/6_/k0_nmk_j3c766qsynb7pr0h40000gp/T/ipykernel_33019/3840281498.py:10: SettingWithCopyWarning: 
A value i

In [5]:
unwanted_families = [pope_count, pope_count + 3, pope_count + 5, pope_count + 6, pope_count + 7] # families which terminate
unwanted_orders = [pope_count + family_count, pope_count + family_count + 3, pope_count + family_count + 6, pope_count + family_count + 9, pope_count + family_count + 12] # orders which terminate

year_nodes = list(years_df['year_node'])
filtered_family_nodes = list(years_df.loc[~years_df['family_node'].isin(unwanted_families), 'family_node'])
sources = year_nodes + [pope_count + 21] * pope_count + filtered_family_nodes  # constructs node sources

family_nodes = list(years_df['family_node'])
filtered_order_nodes = list(years_df.loc[~years_df['order_node'].isin(unwanted_orders), 'order_node'])
targets = [pope_count + 21] * pope_count + family_nodes + filtered_order_nodes # construct node targets

In [6]:
node_labels = list(years_df['names_and_years']) + ['None'] + families[1:] + ['None'] + orders[1:] + ['Popes of the Catholic Church'] # constructs node label list

In [7]:
color_dict = {'None': '#5d5d5d',
 'Order of Saint Benedict': '#7d4881',
 'Canons Regular': '#3d5c68',
 'Cistercian': '#f7dc6f',
 'Premonstratense': '#6492a3',
 'Secular Franciscan Order': '#2d622d',
 'Dominican': '#e59866',
 'Order of Friars Minor': '#469846',
 'Order of Saint Augustine': '#99b7c2',
 'Theatine': '#661212',
 'Order of Friars Minor Conventual': '#70bd70',
 'Camaldolese': '#aa93ad',
 'Jesuit': '#2b1b0b'
}
years_df = years_df.fillna('None')
years_df['color'] = years_df['Order'].map(color_dict)
unwanted_colors = [pope_count + family_count, pope_count + family_count + 3, pope_count + family_count + 6, pope_count + family_count + 9, pope_count + family_count + 12]
filtered_colors = list(years_df.loc[~years_df['order_node'].isin(unwanted_orders), 'color'])
color_list = list(years_df['color']) * 2 + filtered_colors  # constructs list of colors

In [8]:
x_loc_list = [0] * pope_count + [1, 0.7, 0.6, 0.81, 0.55, 0.8, 0.79, 0.78] + [0.87, 0.84, 0.9, 0.9, 0.87, 0.85, 0.85, 0.9]
#[None, 'Benedictine', 'Augustinian', 'Cistercian', 'Franciscan', 'Dominican', 'Theatine', 'Jesuit'] 
y_families = [0.4, 0.62, 0.65, 0.77 ,0.675, 0.79, 0.81, .83]
#['Order of Saint Benedict', 'Canons Regular', 'Premonstratense', 'Secular Franciscan Order', 'Order of Friars Minor', 'Order of Saint Augustine', 'Order of Friars Minor Conventual', 'Camaldolese']
y_orders = [.6, .645, .665, .71, .73, .685, 0.75, .62]
y_loc_list = list(years_df['year_node']/269) + y_families + y_orders

In [9]:
fig = go.Figure(data=[go.Sankey(
    arrangement = 'fixed',
    node = dict(
        x = x_loc_list,
        y = y_loc_list,
        pad = 50,
        thickness = 1,
        line = dict(color = color_list, width = 0.3),
        label = node_labels,
        color = '#0F0F0F',

    ),
    link = dict(
        source = sources,
        target = targets,
        color = color_list,
        value = [0.1] * 3000  # all values are the same. Disrupting this makes plotly reorder the nodes.
        ),
 )])

fig.add_annotation(x=0.755, y=0.78,
            text="Sources:",
            showarrow=False,)
fig.add_annotation(x=0.755, y=0.775,
            text="https://en.wikipedia.org/wiki/List_of_popes",
            showarrow=False)
fig.add_annotation(x=0.595, y=0.77,
            text="https://www.theguardian.com/news/datablog/2013/feb/13/popes-full-list",
            showarrow=False)
fig.add_annotation(x=0.95, y=0,
            text="Made By Justin Tung",
            showarrow=False,)

fig.update_layout(title={
                'text': "Popes by Order",
                'y':0.8,
                'x':0.536,
                 'font':{'size':110},
                'xanchor': 'center',
                'yanchor': 'top'},
                 font_size=17,
                  font_family="serif",
                  title_font_family='TeX Gyre Bonum',
                  font_color="white",
                  paper_bgcolor='grey',
                  width=1700, height=5000, margin_b=100)



fig.show()